# مساعد - متحدثك الذكي

### TTS and STT

A comparison of various Speech-to-Text (STT) tools is conducted in the following notebook: [here](https://colab.research.google.com/drive/1pC9WBNFB361Fp_GBAeEytwPO0y5eD7FW?usp=sharing)

we can use whisper https://github.com/quic/ai-hub-apps/tree/main/apps/windows/python/Whisper

In [ ]:
pip -q install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 22.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 41.2MiB/s]


In [ ]:
# result1 = model.transcribe("I'm Sick.wav")
result2 = model.transcribe("Request.wav",  language="ar")


### Import the LLM

Installing The Required Libraries

In [1]:
!pip install -q transformers accelerate sentencepiece einops

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "humain-ai/ALLaM-7B-Instruct-preview"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

def generate_allam(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text


### Prompots Templates (three prompt for three tasks)

In [3]:
import json

INTENT_SYSTEM_PROMPT = """
انت مساعد ذكي لنظام خدمات حكومية.
مهمتك فقط تصنيف نية المستخدم من الرسالة.
الأنواع المحتملة:
- شرح خدمة
- بدء خدمة جديدة
- استكمال خدمة قائمة
- استفسار عام
- مشكلة تقنية في الدخول

ارجع الناتج ككائن JSON فقط بهذا الشكل:
{"intent": "<اكتب النوع هنا بالضبط من القائمة>"}
"""

def build_intent_prompt(user_message):
    return INTENT_SYSTEM_PROMPT + "\n\nرسالة المستخدم:\n" + user_message


In [4]:
def classify_intent(user_message):
    prompt = build_intent_prompt(user_message)
    raw = generate_allam(prompt, max_new_tokens=64)
    # محاولة استخراج JSON من المخرجات
    start = raw.find("{")
    end = raw.rfind("}")
    if start != -1 and end != -1:
        json_str = raw[start:end+1]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return {"intent": None, "raw": raw}
    return {"intent": None, "raw": raw}


In [5]:
SLOT_SYSTEM_PROMPT = """
انت مساعد استخراج حقول لمحادثة خدمات حكومية.
يُعطى لك نص المستخدم وقائمة بالحقول المتوقعة.
مهمتك:
- استنتاج قيمة كل حقل من النص اذا كانت موجودة
- اذا لم توجد القيمة اجعلها null

ارجع الناتج ككائن JSON بالشكل:
{"slots": {"field1": "value or null", "field2": "value or null", ...}}
"""

def build_slot_prompt(user_message, expected_slots):
    return (
        SLOT_SYSTEM_PROMPT
        + "\n\nالحقول المطلوبة:\n"
        + ", ".join(expected_slots)
        + "\n\nرسالة المستخدم:\n"
        + user_message
    )

def extract_slots(user_message, expected_slots):
    prompt = build_slot_prompt(user_message, expected_slots)
    raw = generate_allam(prompt, max_new_tokens=128)
    start = raw.find("{")
    end = raw.rfind("}")
    if start != -1 and end != -1:
        json_str = raw[start:end+1]
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            return {"slots": {}, "raw": raw}
    return {"slots": {}, "raw": raw}


In [ ]:
RAG_SYSTEM_PROMPT = """
انت مساعد رسمي يشرح خدمات نظام حكومي.
لديك مقاطع نصية رسمية من دليل الخدمات.
استخدم المعلومات الموجودة فقط في هذه المقاطع للاجابة.
لا تضف معلومات من عندك.

اكتب الاجابة بالعربية الواضحة في فقرتين كحد اقصى.
"""

def build_rag_prompt(user_message, context_chunks):
    context_text = "\n\n--- مقاطع من الدليل الرسمي ---\n" + "\n\n".join(context_chunks)
    return RAG_SYSTEM_PROMPT + context_text + "\n\nسؤال المستخدم:\n" + user_message

def rag_answer(user_message, context_chunks):
    prompt = build_rag_prompt(user_message, context_chunks)
    answer = generate_allam(prompt, max_new_tokens=256)
    return answer


In [ ]:
STEP_GUIDE_PROMPT = """
انت مساعد حواري تنفذ خدمة حكومية خطوة بخطوة.
يعطى لك:
- وصف مختصر لمراحل الخدمة
- حالة جلسة المستخدم الحالية
- قائمة بالحقول الناقصة

مهمتك:
- كتابة رسالة قصيرة للمستخدم تسأله عن الحقول الناقصة
- او تشرح له الخطوة التالية بلغة بسيطة
- لا تذكر تفاصيل تقنية او JSON في الرسالة للمستخدم
"""

def build_step_prompt(service_schema, session_state, missing_fields):
    return (
        STEP_GUIDE_PROMPT
        + "\n\nمخطط الخدمة:\n"
        + json.dumps(service_schema, ensure_ascii=False)
        + "\n\nحالة الجلسة:\n"
        + json.dumps(session_state, ensure_ascii=False)
        + "\n\nالحقول الناقصة:\n"
        + ", ".join(missing_fields)
        + "\n\nاكتب رسالة موجهة للمستخدم:"
    )

def step_guide(service_schema, session_state, missing_fields):
    prompt = build_step_prompt(service_schema, session_state, missing_fields)
    reply = generate_allam(prompt, max_new_tokens=192)
    return reply


### ٍRouting Script

In [ ]:
def ai_router(mode, payload):
    if mode == "classify_intent":
        user_message = payload["user_message"]
        return classify_intent(user_message)

    if mode == "extract_slots":
        user_message = payload["user_message"]
        expected_slots = payload["expected_slots"]
        return extract_slots(user_message, expected_slots)

    if mode == "rag_answer":
        user_message = payload["user_message"]
        context_chunks = payload.get("context_chunks", [])
        return {"answer": rag_answer(user_message, context_chunks)}

    if mode == "step_guide":
        service_schema = payload["service_schema"]
        session_state = payload["session_state"]
        missing_fields = payload.get("missing_fields", [])
        return {"reply": step_guide(service_schema, session_state, missing_fields)}

    return {"error": "unknown_mode"}

In [ ]:
res = ai_router("classify_intent", {
    "user_message": "حاولت احدث رقم الجوال وما قدرت ادخل على ابشر ابدا"
})
res


In [ ]:
slots_res = ai_router("extract_slots", {
    "user_message": "انا مسجل في ابشر برقم هوية 1XXXXXXXXX وابغى اغير جوالي القديم",
    "expected_slots": ["national_id", "mobile_number", "service_type"]
})
slots_res
